## Spots that are always excluded:
##### Test spots
##### Spots marked as problematic
##### Suscpicious for analysis == 'Yes' spots
##### Fake Sale spots: This means that Reason for Cancelling Survey is any combination which contains 'Fake Sale'
##### Spots whose stop date was before 2018


## CAN CANCEL:
##### Spots which cancelled more than 2 months before their Available Cancellation Date are excluded
##### Spots whose Available Cancellation Date is 2 or more months after the date of analysis
##### Months during which spots couldn't have cancelled according to the above conditions

## wo CB sets:
##### Closed Business, Sold Business and Non-payment spots are excluded. This means Reason for Cancelling Survey is any combination which contains 'Closed Business', 'Closed/Sold Business', 'Sold Business', 'Sold/Closed Business' or 'Non-payment'
## canc conf event
##### Spots for which Date Cancellation Confirmed is more than 60 days after Date Cancellation Requested

#### ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [1]:
import pandas as pd
import numpy as np
import importlib
import get_stopped_doing_something_variables
import prepare_for_the_models
import fit_tv_cox_models
#import read_vars_for_separate_models_for_dependent_vars
import read_a_combination_of_variables
import yaml
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display, Markdown

In [2]:
# read the yaml file with a list of parameters needed for the report
with open(r'./parameters/stopped_doing_something_report_parameters.yaml') as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)

date_of_analysis = parameters['date_of_analysis']
date_dir = date_of_analysis.replace('-', '_')
### name of the data directory ###
churn_based_on_behaviour_dir = parameters['churn_based_on_behaviour_dir']
### penalizer value when fitting the models ###
penalizer = parameters['penalizer']
### model type ###
model_type = parameters['model_type']
### coefficient and p values when dropping unsignificant variables ###
coefficient_limit_for_numerical_vars = parameters['coefficient_limit_for_numerical_vars']
coefficient_limit_for_cat_vars = parameters['coefficient_limit_for_cat_vars']
p_limit = parameters['p_limit']
additional_higher_p_limit = parameters['additional_higher_p_limit']
additional_lower_p_limit = parameters['additional_lower_p_limit']

# read the yaml file with data set parameters #
# data_set_name = input('Data set name: ')
data_set_name = 'CAN_CANCEL_spots_wo_CB_cancellation_confirmed'
with open(r'./parameters/data_sets.yaml') as file:
    data_sets_parameters = yaml.load(file, Loader=yaml.FullLoader)
    
spots_set = data_sets_parameters[data_set_name]['spots_set']
with_wo_CB = data_sets_parameters[data_set_name]['with_wo_CB']
event_date_full_name = data_sets_parameters[data_set_name]['event_date_type']

if with_wo_CB == 'with_CB':
    with_wo_CB_boolean = True
else:
    with_wo_CB_boolean = False

if event_date_full_name == 'cancellation_requested':
    event_date = 'canc_req'
elif event_date_full_name == 'cancellation_confirmed':
    event_date = 'canc_conf'

In [3]:
##### base columns ##### 
base_cols = ['spot_id',\
            'time',\
            'event']

In [4]:
##### Model 0: all variables - prepare data for the model ####
cols_to_use = read_a_combination_of_variables.\
main(model_number=0, dir_name='combinations_of_variables_that_are_not_dependent/')
#### get behavioural variables ####
(variables_to_use_for_the_model, did_something_before_vars, did_something_last_month_vars) = \
get_stopped_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)
cols = base_cols + variables_to_use_for_the_model + \
did_something_before_vars + did_something_last_month_vars


#### get data for the model ###
(data, base_df, df_timeline_all_vars) = \
prepare_for_the_models.get_data_for_the_MV_Cox_model(date_of_analysis=date_of_analysis, spots_set=spots_set, \
                            with_wo_CB=with_wo_CB, event_date=event_date, columns=cols, data_dir=churn_based_on_behaviour_dir,\
                            C = 100)
    
model_numbers = \
read_a_combination_of_variables.get_a_list_of_model_numbers(dir_name='combinations_of_variables_that_are_not_dependent/')

In [5]:
for model_number in model_numbers[1:]:
    cols_to_use = read_a_combination_of_variables.\
    main(model_number=model_number, dir_name='combinations_of_variables_that_are_not_dependent/')
    
    model_name = \
    read_a_combination_of_variables.get_model_names(model_number=model_number, \
                                                    dir_name='combinations_of_variables_that_are_not_dependent/')
    display(Markdown("# Model "+ str(model_number) + ": " + model_name))

    #### get behavioural variables ####
    (variables_to_use_for_the_model, did_something_before_vars, did_something_last_month_vars) = \
    get_stopped_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)
    cols = variables_to_use_for_the_model + \
    did_something_before_vars + did_something_last_month_vars

    #### data for the model ###
    df_timeline = df_timeline_all_vars.copy()
    vars_that_stay = ['spot_id', 'start', 'stop', 'event']+\
    [x for x in cols if x not in base_cols]+\
    [x for x in df_timeline.columns if 'spot_category_' in x or 'metro_area_' in x]
    df_timeline.drop([x for x in df_timeline.columns if x not in vars_that_stay], axis = 1, inplace = True)

    ### variables to skip ###
    df_timeline.isnull().sum().sum() #OK
    skip_vars = list((df_timeline!=0).sum()[(df_timeline!=0).sum()==0].index)
    skip_vars

    ctv = fit_tv_cox_models.fit_the_models_and_print_summaries(df_timeline=df_timeline, base_df=base_df, \
                                                         date_of_analysis=date_of_analysis, model_type=model_type, \
                                                         variables_to_use_for_the_model=cols_to_use,\
                                                         coefficient_limit_for_numerical_vars=coefficient_limit_for_numerical_vars, \
                                                         coefficient_limit_for_cat_vars=coefficient_limit_for_cat_vars, p_limit=p_limit, \
                                                         additional_higher_p_limit=additional_higher_p_limit, \
                                                         additional_lower_p_limit=additional_lower_p_limit,\
                                                         skip_vars=skip_vars, penalizer=penalizer)

    ### save coefs and p values for p < 0.2 ###
    fit_tv_cox_models.save_results(df = ctv.summary.reset_index(), date_of_analysis = date_of_analysis, \
                                   data_dir = churn_based_on_behaviour_dir, dir_name='exports/coefficients_and_pvalues',\
                 results_name = 'coef_and_pvalues', spots_set = spots_set,\
                 with_wo_CB = with_wo_CB, event_date_type = event_date_full_name, p_limit=0.2, model_number=model_number,\
                                  model_type = model_type)

# Model 1: posts_on_facebook

## without variables with p >= 0.2

### sorted by variable name

<lifelines.CoxTimeVaryingFitter: fitted with 16618 periods, 1356 subjects, 147 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 1356
 number of periods = 16618
  number of events = 147
partial log-likelihood = -847.82
  time fit was run = 2021-12-21 10:01:54 UTC

---
                                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                  
Annual Prepay                                        0.55       1.73       0.19             0.18             0.91                 1.20                 2.49
Monthly                                              0.55       1.73       0.38            -0.20             1.30                 0.82                 3.68
Website.Views.last.month.total.log2                 -0.18       0.84       0.06            -0.29            -0.06                 0.75                 0.94
had_added_events_manually_edited_events_before      -0.03       0.97       0.21            -0.43             0.37                 0.65                 1.45
had_added_events_manually_edited_events_last_3_...  -0.65       0.52       0.37            -1.39             0.08                 0.25                 1.08
had_catering_submissions_before                      0.01       1.01       0.25            -0.49             0.51                 0.62                 1.66
had_catering_submissions_last_3_months              -0.45       0.64       0.31            -1.05             0.16                 0.35                 1.18
had_clicked_emails_before                            0.99       2.70       0.26             0.49             1.49                 1.63                 4.46
had_clicked_emails_last_3_months                    -0.58       0.56       0.20            -0.98            -0.18                 0.37                 0.83
had_emails_sent_manually_scheduled_emails_sent_...   0.06       1.06       0.19            -0.33             0.44                 0.72                 1.55
had_emails_sent_manually_scheduled_emails_sent_...  -0.50       0.61       0.23            -0.95            -0.05                 0.39                 0.96
had_online_orders_before                             0.51       1.67       0.33            -0.14             1.16                 0.87                 3.19
had_online_orders_last_month                        -0.68       0.51       0.46            -1.59             0.23                 0.20                 1.26
had_posts_on_facebook_before                         0.64       1.89       0.22             0.21             1.07                 1.23                 2.92
had_posts_on_facebook_last_month                    -0.42       0.66       0.19            -0.80            -0.05                 0.45                 0.95
had_qr_code_menu_scans_before                       -0.50       0.61       0.30            -1.08             0.09                 0.34                 1.09
had_qr_code_menu_scans_last_month                   -0.00       1.00       0.33            -0.65             0.65                 0.52                 1.91
had_reservations_submissions_before                  0.10       1.11       0.28            -0.45             0.66                 0.64                 1.93
had_reservations_submissions_last_month             -0.63       0.53       0.44            -1.50             0.23                 0.22                 1.26
had_tickets_before                                  -1.04       0.35       0.21            -1.46            -0.62                 0.23                 0.54
had_tickets_last_3_months                            0.51       1.66       0.20             0.11             0.90                 1.12                 2.46
had_visited_events_page_before                       0.33       1.40       0.24            -0.14            

# Model 2: changed_picture_or_text

...

<lifelines.CoxTimeVaryingFitter: fitted with 16618 periods, 1356 subjects, 147 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 1356
 number of periods = 16618
  number of events = 147
partial log-likelihood = -834.38
  time fit was run = 2021-12-21 11:06:13 UTC

---
                                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                  
Annual Prepay                                        0.56       1.75       0.19             0.19             0.93                 1.21                 2.54
Monthly                                              0.58       1.78       0.39            -0.18             1.33                 0.84                 3.79
Opted.out.of.facebook                                0.54       1.71       0.28            -0.01             1.09                 0.99                 2.96
Website.Views.last.month.total.log2                 -0.20       0.82       0.06            -0.32            -0.08                 0.72                 0.92
had_added_events_manually_edited_events_before       0.01       1.01       0.20            -0.39             0.41                 0.68                 1.51
had_added_events_manually_edited_events_last_3_...  -0.63       0.53       0.36            -1.33             0.07                 0.27                 1.08
had_catering_submissions_before                     -0.02       0.98       0.26            -0.52             0.48                 0.60                 1.62
had_catering_submissions_last_3_months              -0.44       0.64       0.31            -1.05             0.16                 0.35                 1.18
had_clicked_emails_before                            0.91       2.48       0.26             0.40             1.42                 1.49                 4.12
had_clicked_emails_last_3_months                    -0.49       0.61       0.21            -0.90            -0.08                 0.41                 0.92
had_emails_sent_manually_scheduled_emails_sent_...   0.09       1.10       0.20            -0.29             0.48                 0.74                 1.62
had_emails_sent_manually_scheduled_emails_sent_...  -0.45       0.64       0.23            -0.90             0.01                 0.41                 1.01
had_number_of_requests_for_new_text_fragment_be...   0.43       1.54       0.20             0.04             0.82                 1.05                 2.27
had_number_of_requests_for_new_text_fragment_la...   0.46       1.59       0.38            -0.27             1.20                 0.76                 3.33
had_online_orders_before                             0.41       1.51       0.33            -0.23             1.06                 0.79                 2.88
had_online_orders_last_month                        -0.65       0.52       0.46            -1.56             0.26                 0.21                 1.30
had_opened_rewarding_stats_emails_before             0.42       1.53       0.33            -0.22             1.07                 0.80                 2.91
had_opened_rewarding_stats_emails_last_2_months     -0.20       0.82       0.38            -0.95             0.55                 0.39                 1.74
had_posts_liked_before                               0.30       1.35       0.21            -0.11             0.71                 0.90                 2.04
had_posts_liked_last_3_months                       -0.74       0.48       0.30            -1.32            -0.15                 0.27                 0.86
had_posts_on_facebook_before                         0.52       1.68       0.25             0.04             1.00                 1.04                 2.71
had_posts_on_facebook_last_month                    -0.34       0.71       0.20            -0.73            

# Model 33: New.email.subscribers.allinclusive.last.month.div10

## without variables with p >= 0.2

### sorted by variable name

<lifelines.CoxTimeVaryingFitter: fitted with 16618 periods, 1356 subjects, 147 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 1356
 number of periods = 16618
  number of events = 147
partial log-likelihood = -839.04
  time fit was run = 2021-12-21 11:07:54 UTC

---
                                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                  
Annual Prepay                                        0.56       1.75       0.19             0.19             0.93                 1.21                 2.53
Monthly                                              0.59       1.81       0.39            -0.16             1.35                 0.85                 3.85
Opted.out.of.facebook                                0.53       1.69       0.28            -0.02             1.07                 0.98                 2.92
Website.Views.last.month.total.log2                 -0.22       0.80       0.06            -0.34            -0.11                 0.71                 0.90
had_added_events_manually_edited_events_before      -0.02       0.98       0.20            -0.42             0.37                 0.66                 1.45
had_added_events_manually_edited_events_last_3_...  -0.70       0.50       0.35            -1.39            -0.00                 0.25                 1.00
had_clicked_emails_before                            0.91       2.50       0.26             0.41             1.42                 1.50                 4.15
had_clicked_emails_last_3_months                    -0.51       0.60       0.21            -0.92            -0.10                 0.40                 0.90
had_emails_sent_manually_scheduled_emails_sent_...   0.07       1.07       0.20            -0.32             0.46                 0.73                 1.58
had_emails_sent_manually_scheduled_emails_sent_...  -0.48       0.62       0.23            -0.93            -0.02                 0.40                 0.98
had_number_of_requests_for_new_text_fragment_be...   0.41       1.51       0.19             0.03             0.79                 1.03                 2.21
had_number_of_requests_for_new_text_fragment_la...   0.44       1.56       0.38            -0.30             1.18                 0.74                 3.26
had_online_orders_before                             0.40       1.49       0.33            -0.24             1.04                 0.79                 2.83
had_online_orders_last_month                        -0.69       0.50       0.46            -1.60             0.21                 0.20                 1.24
had_posts_liked_before                               0.30       1.36       0.21            -0.11             0.71                 0.90                 2.04
had_posts_liked_last_3_months                       -0.74       0.48       0.30            -1.33            -0.16                 0.26                 0.85
had_posts_on_facebook_before                         0.53       1.70       0.25             0.05             1.01                 1.05                 2.74
had_posts_on_facebook_last_month                    -0.37       0.69       0.20            -0.75             0.02                 0.47                 1.02
had_posts_seen_before                                0.27       1.31       0.29            -0.30             0.84                 0.74                 2.33
had_posts_seen_last_3_months                        -0.45       0.64       0.21            -0.86            -0.04                 0.42                 0.96
had_qr_code_menu_scans_before                       -0.50       0.61       0.30            -1.09             0.09                 0.34                 1.10
had_qr_code_menu_scans_last_month                   -0.04       0.96       0.33            -0.69            